In [0]:

# MAGIC %md
# MAGIC # 05 - Model Deployment & Serving
# MAGIC 
# MAGIC **Deploy the best model to production and create serving endpoints**
# MAGIC 
# MAGIC ## Objectives:
# MAGIC - Transition model to Production stage
# MAGIC - Create model serving endpoint
# MAGIC - Test predictions with sample data
# MAGIC - Document deployment process
# MAGIC - Setup monitoring


In [0]:

# MAGIC %md
# MAGIC ## 1. Setup & Imports


In [0]:
%restart_python


In [0]:

# Standard imports
import sys
import numpy as np
import pandas as pd
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# MLflow
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# Databricks
from databricks import feature_store

print("✅ Imports complete")


In [0]:
# MAGIC %md
# MAGIC ## 2. Project Setup


In [0]:
print("="*60)
print("PROJECT SETUP")
print("="*60)

# Define project root
project_root = "/Workspace/COMM - Commercial Analytics (CMAN)/MMM Quattro 2025/Satish/MLFLOW_sample"

# Add to path
if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"\n📂 Project root: {project_root}")
print(f"✅ Added to sys.path")

# Import custom modules
from src.utils import ConfigLoader, DataLoader, MLflowLogger, safe_display

print(f"✅ Custom modules imported")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 3. Load Configuration


In [0]:

print("="*60)
print("LOADING CONFIGURATION")
print("="*60)

config_path = f'{project_root}/config/config.yaml'
config = ConfigLoader.load_config(config_path)

print(f"\n✅ Configuration loaded")
print(f"  • Project: {config['project']['name']}")
print(f"  • Model Registry: {config['mlflow']['model_registry_name']}")
print(f"  • Deployment Stage: {config['deployment']['stage']}")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 4. Setup MLflow


In [0]:

print("="*60)
print("SETTING UP MLFLOW")
print("="*60)

experiment_name = config['mlflow']['experiment_name']
mlflow.set_experiment(experiment_name)

# Initialize MLflow client
client = MlflowClient()

print(f"\n✅ MLflow experiment set: {experiment_name}")

experiment = mlflow.get_experiment_by_name(experiment_name)
print(f"  • Experiment ID: {experiment.experiment_id}")
print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 5. Get Model Information


In [0]:

print("="*60)
print("RETRIEVING MODEL INFORMATION")
print("="*60)

model_name = config['mlflow']['model_registry_name']

try:
    # Get all versions of the model
    model_versions = client.search_model_versions(f"name='{model_name}'")
    
    if not model_versions:
        raise ValueError(f"No versions found for model: {model_name}")
    
    # Get latest version
    latest_version = model_versions[0]
    
    print(f"\n📦 Model: {model_name}")
    print(f"  • Latest Version: {latest_version.version}")
    print(f"  • Current Stage: {latest_version.current_stage}")
    print(f"  • Run ID: {latest_version.run_id}")
    print(f"  • Status: {latest_version.status}")
    
    # Get run details
    run = client.get_run(latest_version.run_id)
    
    print(f"\n📊 Model Metrics:")
    metrics = run.data.metrics
    for metric_name, metric_value in sorted(metrics.items()):
        if 'test' in metric_name:
            print(f"  • {metric_name}: {metric_value:.4f}")
    
    print(f"\n📋 Total Versions: {len(model_versions)}")
    
except Exception as e:
    print(f"\n❌ Error retrieving model: {e}")
    print(f"\n💡 Tip: Make sure you ran the model training notebook first!")
    raise

print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 6. Transition Model to Production


In [0]:

print("="*60)
print("TRANSITIONING MODEL TO PRODUCTION")
print("="*60)

target_stage = config['deployment']['stage']

try:
    # Check current stage
    current_stage = latest_version.current_stage
    
    print(f"\n📋 Current Stage: {current_stage}")
    print(f"🎯 Target Stage: {target_stage}")
    
    if current_stage == target_stage:
        print(f"\n✅ Model is already in {target_stage} stage")
    else:
        # Transition to target stage
        client.transition_model_version_stage(
            name=model_name,
            version=latest_version.version,
            stage=target_stage,
            archive_existing_versions=True  # Archive old production versions
        )
        
        print(f"\n✅ Model transitioned successfully!")
        print(f"  • Model: {model_name}")
        print(f"  • Version: {latest_version.version}")
        print(f"  • Stage: {current_stage} → {target_stage}")
        print(f"  • Old versions archived: Yes")
    
    # Add description/tags
    client.update_model_version(
        name=model_name,
        version=latest_version.version,
        description=f"Production model for house price prediction. Deployed on {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}"
    )
    
    print(f"\n📝 Model description updated")
    
except Exception as e:
    print(f"\n❌ Error transitioning model: {e}")
    import traceback
    traceback.print_exc()
    raise

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 7. Load Production Model


In [0]:

print("="*60)
print("LOADING PRODUCTION MODEL")
print("="*60)

try:
    # Load model from Production stage
    model_uri = f"models:/{model_name}/{target_stage}"
    
    print(f"\n📦 Loading model: {model_uri}")
    
    production_model = mlflow.sklearn.load_model(model_uri)
    
    print(f"\n✅ Production model loaded successfully!")
    print(f"  • Model Name: {model_name}")
    print(f"  • Model Type: {type(production_model).__name__}")
    print(f"  • Stage: {target_stage}")
    
except Exception as e:
    print(f"\n❌ Error loading production model: {e}")
    raise

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 8. Test Predictions


In [0]:
# COMMAND ----------

# MAGIC %md
# MAGIC ## 8. Test Predictions

# COMMAND ----------

print("="*60)
print("TESTING PRODUCTION MODEL")
print("="*60)

# Load test data
processed_path = config['data']['processed_path']
X_test = pd.read_csv(f"{processed_path}X_test.csv")
y_test = pd.read_csv(f"{processed_path}y_test.csv").squeeze()

print(f"\n📊 Test data loaded: {X_test.shape}")

# Create sample data for testing - dynamically based on test set size
n_samples = min(5, len(X_test))  # Use 5 samples or less if test set is smaller
sample_indices = np.linspace(0, len(X_test)-1, n_samples, dtype=int)

X_sample = X_test.iloc[sample_indices]
y_sample = y_test.iloc[sample_indices]

print(f"\n🧪 Testing with {len(sample_indices)} samples...")
print(f"  • Sample indices: {list(sample_indices)}")

# Make predictions
predictions = production_model.predict(X_sample)

# Create results DataFrame
test_results = pd.DataFrame({
    'Actual_Price': y_sample.values,
    'Predicted_Price': predictions,
    'Difference': y_sample.values - predictions,
    'Abs_Error': np.abs(y_sample.values - predictions),
    'Pct_Error': np.abs((y_sample.values - predictions) / y_sample.values) * 100
})

print(f"\n📊 Sample Predictions:")
safe_display(test_results)

# Calculate metrics on sample
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

sample_metrics = {
    'RMSE': np.sqrt(mean_squared_error(y_sample, predictions)),
    'MAE': mean_absolute_error(y_sample, predictions),
    'R2': r2_score(y_sample, predictions)
}

print(f"\n📈 Sample Metrics:")
print(f"  • RMSE: ${sample_metrics['RMSE']:,.2f}")
print(f"  • MAE:  ${sample_metrics['MAE']:,.2f}")
print(f"  • R²:   {sample_metrics['R2']:.4f}")

print("="*60)

In [0]:

# MAGIC %md
# MAGIC ## 10. Batch Prediction Example


In [0]:

print("="*60)
print("BATCH PREDICTION EXAMPLE")
print("="*60)

# Load full test set
print(f"\n📊 Running batch predictions on full test set...")
print(f"  • Test set size: {len(X_test)}")

# Make predictions
batch_predictions = production_model.predict(X_test)

# Create results DataFrame
batch_results = pd.DataFrame({
    'Actual_Price': y_test.values,
    'Predicted_Price': batch_predictions,
    'Difference': y_test.values - batch_predictions,
    'Abs_Error': np.abs(y_test.values - batch_predictions),
    'Pct_Error': np.abs((y_test.values - batch_predictions) / y_test.values) * 100
})

# Calculate metrics
batch_metrics = {
    'RMSE': np.sqrt(mean_squared_error(y_test, batch_predictions)),
    'MAE': mean_absolute_error(y_test, batch_predictions),
    'R2': r2_score(y_test, batch_predictions),
    'Mean_Pct_Error': batch_results['Pct_Error'].mean(),
    'Median_Pct_Error': batch_results['Pct_Error'].median()
}

print(f"\n✅ Batch predictions complete")

print(f"\n📈 Batch Metrics:")
print(f"  • RMSE:              ${batch_metrics['RMSE']:,.2f}")
print(f"  • MAE:               ${batch_metrics['MAE']:,.2f}")
print(f"  • R²:                {batch_metrics['R2']:.4f}")
print(f"  • Mean % Error:      {batch_metrics['Mean_Pct_Error']:.2f}%")
print(f"  • Median % Error:    {batch_metrics['Median_Pct_Error']:.2f}%")

print(f"\n📊 Prediction Summary:")
safe_display(batch_results.describe())

# Save batch predictions
batch_predictions_path = f"{processed_path}production_predictions.csv"
batch_results.to_csv(batch_predictions_path, index=False)
print(f"\n💾 Batch predictions saved: {batch_predictions_path}")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 11. Model Serving Information


In [0]:

print("="*60)
print("MODEL SERVING INFORMATION")
print("="*60)

print(f"\n📦 Production Model Details:")
print(f"  • Model Name: {model_name}")
print(f"  • Version: {latest_version.version}")
print(f"  • Stage: {target_stage}")
print(f"  • Model Type: {type(production_model).__name__}")

print(f"\n🔗 MLflow Model URI:")
print(f"  • URI: models:/{model_name}/{target_stage}")
print(f"  • Run ID: {latest_version.run_id}")

print(f"\n📊 Model Performance:")
print(f"  • RMSE: ${batch_metrics['RMSE']:,.2f}")
print(f"  • MAE:  ${batch_metrics['MAE']:,.2f}")
print(f"  • R²:   {batch_metrics['R2']:.4f}")

print(f"\n📋 Input Features ({len(X_test.columns)}):")
for i, col in enumerate(X_test.columns, 1):
    print(f"  {i}. {col}")

print(f"\n💡 Usage Example (Python):")
print(f"""
import mlflow

# Load model
model = mlflow.sklearn.load_model("models:/{model_name}/{target_stage}")

# Make prediction
prediction = model.predict(input_data)
""")

print(f"\n💡 Usage Example (REST API):")
print(f"""
# Note: You would need to create a Model Serving Endpoint in Databricks
# Go to: Machine Learning → Serving → Create Serving Endpoint
# Select model: {model_name}
# Version: {latest_version.version}
""")

print("="*60)


In [0]:
# MAGIC %md
# MAGIC ## 12. Save Deployment Documentation


In [0]:

print("="*60)
print("SAVING DEPLOYMENT DOCUMENTATION")
print("="*60)

# Create deployment documentation
deployment_doc = {
    'deployment_info': {
        'model_name': model_name,
        'model_version': latest_version.version,
        'model_stage': target_stage,
        'model_type': type(production_model).__name__,
        'deployment_date': pd.Timestamp.now().isoformat(),
        'deployed_by': config['project']['author']
    },
    'model_performance': {
        'rmse': float(batch_metrics['RMSE']),
        'mae': float(batch_metrics['MAE']),
        'r2': float(batch_metrics['R2']),
        'mean_pct_error': float(batch_metrics['Mean_Pct_Error']),
        'median_pct_error': float(batch_metrics['Median_Pct_Error'])
    },
    'model_uri': {
        'production': f"models:/{model_name}/{target_stage}",
        'versioned': f"models:/{model_name}/{latest_version.version}",
        'run_id': latest_version.run_id
    },
    'input_features': list(X_test.columns),
    'target_variable': config['preprocessing']['target'],
    'test_set_size': len(X_test)
}

# Save as JSON
deployment_doc_path = f"{processed_path}deployment_documentation.json"
with open(deployment_doc_path, 'w') as f:
    json.dump(deployment_doc, f, indent=2)

print(f"\n✅ Deployment documentation saved: {deployment_doc_path}")

# Also save as readable text
deployment_txt_path = f"{processed_path}deployment_summary.txt"
with open(deployment_txt_path, 'w') as f:
    f.write("="*60 + "\n")
    f.write("MODEL DEPLOYMENT SUMMARY\n")
    f.write("="*60 + "\n\n")
    
    f.write(f"Model Name: {model_name}\n")
    f.write(f"Version: {latest_version.version}\n")
    f.write(f"Stage: {target_stage}\n")
    f.write(f"Model Type: {type(production_model).__name__}\n")
    f.write(f"Deployment Date: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    
    f.write("Performance Metrics:\n")
    f.write(f"  • RMSE: ${batch_metrics['RMSE']:,.2f}\n")
    f.write(f"  • MAE:  ${batch_metrics['MAE']:,.2f}\n")
    f.write(f"  • R²:   {batch_metrics['R2']:.4f}\n\n")
    
    f.write(f"Model URI: models:/{model_name}/{target_stage}\n")
    f.write(f"Run ID: {latest_version.run_id}\n\n")
    
    f.write(f"Input Features ({len(X_test.columns)}):\n")
    for i, col in enumerate(X_test.columns, 1):
        f.write(f"  {i}. {col}\n")

print(f"✅ Deployment summary saved: {deployment_txt_path}")

print("="*60)


In [0]:

# MAGIC %md
# MAGIC ## 13. Deployment Summary


In [0]:

print("="*60)
print("DEPLOYMENT COMPLETE!")
print("="*60)

print(f"\n🎉 Model successfully deployed to {target_stage}!")

print(f"\n📦 Model Information:")
print(f"  • Name: {model_name}")
print(f"  • Version: {latest_version.version}")
print(f"  • Type: {type(production_model).__name__}")
print(f"  • Stage: {target_stage}")

print(f"\n📊 Performance:")
print(f"  • RMSE: ${batch_metrics['RMSE']:,.2f}")
print(f"  • MAE:  ${batch_metrics['MAE']:,.2f}")
print(f"  • R²:   {batch_metrics['R2']:.4f}")

print(f"\n📁 Generated Files:")
print(f"  • production_predictions.csv")
print(f"  • deployment_documentation.json")
print(f"  • deployment_summary.txt")

print(f"\n🔗 Access Model:")
print(f"  • MLflow UI: Experiments → {experiment_name}")
print(f"  • Model Registry: Models → {model_name}")
print(f"  • Model URI: models:/{model_name}/{target_stage}")

print(f"\n📝 Next Steps:")
print(f"  1. Review deployment documentation")
print(f"  2. Create Model Serving Endpoint (optional)")
print(f"  3. Setup monitoring and alerts")
print(f"  4. Document API usage for consumers")
print(f"  5. Schedule periodic model retraining")

print(f"\n✅ MLOps Pipeline Complete!")
print("="*60)